In [1]:
import os
os.chdir("/home/valentin/Desktop/TagThePicture/presentation_jan_2026/TagThePicture_data/")
from main import get_annotations, get_thesaurus, get_hierarchy, reverse_prefix, pd

from treelib import Node, Tree
import graphviz 
from graphviz import Source

In [2]:
anns = get_annotations()

# terms = anns.termIdentifier.value_counts().index

some_terms = anns.termIdentifier.value_counts().sample(10).index
# thes = get_thesaurus(terms)

---

## visualising the part of the thesaurus used in TTP

In [4]:
above = []

cur = some_terms
for _ in range(10):
    if sorted(cur.unique()) == ['https://hdl.handle.net/20.500.11840/termmaster1']:
        continue

    cur = get_hierarchy(sorted(cur.unique())).broader.apply(reverse_prefix)
    cur = cur.reset_index().drop_duplicates().set_index("conceptURI").broader
    above.append(cur)
above = above[::-1]

all_terms = sorted(set(uri for srs in above for row in zip(srs.index, srs) for uri in row))
all_terms_thes = get_thesaurus(all_terms).drop_duplicates()
english = all_terms_thes[all_terms_thes.altLabelLanguage == "en"]
labels = english.altLabel.drop_duplicates()

In [5]:
tree = Tree()
tree.create_node(":", "root")
above[0].apply(lambda b: tree.create_node(labels.loc[b] if b in labels.index else ":", b, parent="root"))
if not "https://hdl.handle.net/20.500.11840/termmaster1" in tree:
    tree.create_node(":", "https://hdl.handle.net/20.500.11840/termmaster1", parent="root")

def linebreak(s):
    if len(s) > 30:
        return s[:15]+"\n"+s[15:]
    else: return s

for srs in above:
    for cur, b in zip(srs.index, srs):
        if cur in tree:
            continue
        try:
            lbl = labels.loc[cur]
            tree.create_node(linebreak(lbl), cur, parent=b)
        except KeyError:
            print("NOT IN THESAURUS:", cur, b)
            tree.create_node(":", cur, parent=b)

tree.to_graphviz("tree.dot", graph="digraph")

with open("tree.dot") as handle:
    dot_str = handle.read()
    s = Source(dot_str, filename="test.gv", format="png")
    s.view()

NOT IN THESAURUS: https://hdl.handle.net/20.500.11840/termmaster10079406 https://hdl.handle.net/20.500.11840/termmaster10066761
NOT IN THESAURUS: https://hdl.handle.net/20.500.11840/termmaster10079410 https://hdl.handle.net/20.500.11840/termmaster10066316



(viewnior:327897): Gtk-WARNING **: 01:14:21.567: Theme parsing error: gtk-contained-dark.css:2871:228: Missing closing bracket for :not()


In [ ]:
with open("tree.dot") as handle:
    dot_str = handle.read()
    s = Source(dot_str, filename="test.gv", format="png")
    s.view()

In [ ]:
s.save